In [ ]:
import pandas as pd
import numpy as np
Candledf = pd.read_csv('../result.dat',low_memory=False)
Candledf['ndatetime']=pd.to_datetime(Candledf['ndatetime'],format='%Y-%m-%d %H:%M:%S.%f')
print(Candledf.tail())


In [ ]:
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm

In [ ]:
Candledf.close.shift(-1)